In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
%matplotlib inline

Creacion de los sets de tests y features. Voy a probar primero con todo junto

In [2]:
df_palabras = pd.read_csv('../features/palabras_clave.csv')
df_word2vec = pd.read_csv('../features/word2vec_vec.csv')
df_features = df_palabras.join(df_word2vec,how = 'inner')

df_train = pd.read_csv('../dataset/train.csv', index_col='id')
df_test = pd.read_csv('../dataset/test.csv', index_col='id')

In [3]:
features_cols = df_features.columns.tolist()
df_train = df_train.join(df_features, how='inner')

X = df_train[features_cols]
Y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=17)

In [4]:
param_grid = { 
    'n_estimators': [x for x in range(100, 500, 100)],
    'max_depth': [3, 5, 10],
    'colsample_bytree': [0.5, 0.7, 0.9]
}

In [5]:
xgb_classifier = xgb.XGBClassifier(random_state=17, learning_rate=0.05)
CV_xgb = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='f1', n_jobs=4)
CV_xgb.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.05, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=17, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'colsample_bytree': [0.5, 0.7, 0.9],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300,

In [6]:
CV_xgb.best_params_

{'colsample_bytree': 0.7, 'max_depth': 10, 'n_estimators': 400}

In [7]:
def try_classifier(classifier):
    predictions = classifier.predict(X_test)
    predictions_with_prob = classifier.predict_proba(X_test)[:,1]
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(f'Verdaderos Negativos: {tn}')
    print(f'Falsos Negativos: {fn}')
    print(f'Verdaderos Positivos: {tp}')
    print(f'Falsos Positivos: {fp}')
    print()
    print()
    print(f'accuracy score: {accuracy_score(y_test, predictions)}')
    print(f'roc auc score: {roc_auc_score(y_test,  predictions_with_prob)}')
    print(f'f1 score: {f1_score(y_test,  predictions)}')

In [8]:
try_classifier(CV_xgb)

Verdaderos Negativos: 588
Falsos Negativos: 431
Verdaderos Positivos: 236
Falsos Positivos: 268


accuracy score: 0.541037426132633
roc auc score: 0.5497047037229049
f1 score: 0.40307429547395385


In [9]:
df_submit = df_test.join(df_features, how='inner')[features_cols]
df_submit.shape

(3263, 305)

In [10]:
kaggle_pred = CV_xgb.predict(df_submit)
len(kaggle_pred)

3263

In [11]:
df_submit['target'] = kaggle_pred

In [12]:
df_submit.reset_index(inplace = True)
df_submit.rename(columns={'index':'id'}, inplace=True)
df_submit =df_submit[['id','target']]

In [13]:
df_submit[['id', 'target']].to_csv('../submits/submit_word2vec.csv',index=False)

In [14]:
df =  pd.read_csv('../submits/submit_word2vec.csv')
df.columns

Index(['id', 'target'], dtype='object')

Cambiemos el modelo a RF.

In [15]:
df_train

,keyword,location,text,target,mencion_divina,contiene_risa,contiene_hora_especifica,mencion_personal,Uso_infinitivos,0,...,290,291,292,293,294,295,296,297,298,299
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,False,False,False,False,False,0.266238,...,0.043816,-0.131888,-0.131770,0.127149,-0.084561,0.229661,0.242890,-0.153219,0.235863,0.109503
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,False,False,False,False,0.226661,...,0.034778,-0.112177,-0.111213,0.108170,-0.069639,0.200038,0.210107,-0.133071,0.205323,0.092914
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,False,False,False,False,False,0.262429,...,0.042872,-0.131888,-0.131770,0.125749,-0.084561,0.228310,0.240722,-0.153219,0.233802,0.107875
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,False,False,False,False,True,0.270674,...,0.034881,-0.134677,-0.135531,0.140075,-0.084350,0.236067,0.242613,-0.154561,0.241616,0.109106
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,False,False,False,True,False,0.271104,...,0.044661,-0.131888,-0.131770,0.128849,-0.084561,0.234542,0.247456,-0.153219,0.240994,0.111180
8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,False,False,False,False,True,0.309327,...,0.047865,-0.154431,-0.152669,0.145025,-0.096448,0.270486,0.283243,-0.178415,0.276549,0.126310
10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,False,False,False,False,False,0.333390,...,0.052074,-0.123700,-0.122763,0.156637,-0.078441,0.289851,0.302914,-0.143529,0.298831,0.135000
13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,False,False,False,False,False,0.250900,...,0.037470,-0.113651,-0.110948,0.118318,-0.071044,0.216875,0.227792,-0.132668,0.222803,0.101584
14,NaN,NaN,There's an emergency evacuation happening now ...,1,False,False,False,False,True,0.337746,...,0.051834,-0.130326,-0.126970,0.163736,-0.081606,0.298595,0.314796,-0.150320,0.304789,0.141282
15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,False,False,False,False,False,0.286263,...,0.044158,-0.103868,-0.102772,0.135520,-0.064646,0.247344,0.261752,-0.122221,0.255686,0.117227


In [48]:
full_model = df_train
model = full_model[full_model['target'].notnull()]
labels = model['target']
model = model.drop('target', axis=1)
model = model.drop(['text','location','keyword'],axis=1)
model

,mencion_divina,contiene_risa,contiene_hora_especifica,similitud_debris,similitud_derailment,similitud_wreckage,similitud_outbreak,similitud_typhoon,similitud_bombing
1,False,False,False,0.713557,0.713932,0.712962,0.711161,0.713982,0.712836
4,False,False,False,0.799072,0.799559,0.798390,0.796361,0.799600,0.798408
5,False,False,False,0.798885,0.799466,0.798186,0.796256,0.799495,0.798310
6,False,False,False,0.915290,0.915863,0.914501,0.912239,0.915872,0.914474
7,False,False,False,0.832373,0.832956,0.831690,0.829543,0.833011,0.831625
8,False,False,False,0.999065,0.999632,0.998188,0.995703,0.999686,0.998170
10,False,False,False,0.999085,0.999715,0.998207,0.995588,0.999759,0.998349
13,False,False,False,0.665921,0.666357,0.665366,0.663734,0.666395,0.665464
14,False,False,False,0.776966,0.777497,0.776310,0.774426,0.777498,0.776476
15,False,False,False,0.999216,0.999730,0.998375,0.995902,0.999747,0.998186


In [56]:
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
rfc=RandomForestClassifier(random_state=42)
# Instantiate the grid search model
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'

In [50]:
CV_rfc.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 2,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 200}

In [51]:
clf1 = RandomForestClassifier(bootstrap= True,
 max_depth= 100,
 max_features = 2,
 min_samples_leaf= 5,
 min_samples_split= 12,
 n_estimators= 200)

In [52]:
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=100, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=12,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [53]:
result = clf1.predict_proba(X_test)

In [54]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [55]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.5398658731381972